In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50, VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Step 1: Create labels from subfolders
image_dir = "images"  # <-- renamed folder
data = []

for age_folder in sorted(os.listdir(image_dir)):
    folder_path = os.path.join(image_dir, age_folder)
    if os.path.isdir(folder_path):
        for img_name in os.listdir(folder_path):
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                data.append({
                    'filepath': os.path.join(age_folder, img_name),
                    'age': int(age_folder)
                })

df = pd.DataFrame(data)

# Parameters
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Step 2: Preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_dir,
    x_col="filepath",
    y_col="age",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=image_dir,
    x_col="filepath",
    y_col="age",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='raw',
    subset="validation",
    shuffle=False
)

# Step 3: Model builder
def build_model(base_model):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(1, activation='linear')(x)  # Regression
    return Model(inputs=base_model.input, outputs=predictions)

def prepare_model(base_fn):
    base_model = base_fn(include_top=False, weights='imagenet', input_shape=(*IMG_SIZE, 3))
    for layer in base_model.layers:
        layer.trainable = False
    model = build_model(base_model)
    model.compile(optimizer=Adam(1e-4), loss='mse', metrics=['mae'])
    return model

# Step 4: Initialize models
models = {
    "ResNet50": prepare_model(ResNet50),
    "VGG16": prepare_model(VGG16)
}

# Step 5: Train and record
histories = {}

for name, model in models.items():
    print(f"\nTraining {name}...")
    history = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS
    )
    histories[name] = history

# Step 6: Plot results
def plot_history(history, title):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f"{title} - Loss")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['mae'], label='Train MAE')
    plt.plot(history.history['val_mae'], label='Val MAE')
    plt.title(f"{title} - MAE")
    plt.legend()
    plt.tight_layout()
    plt.show()

for name, history in histories.items():
    plot_history(history, name)




ModuleNotFoundError: No module named 'tensorflow'